In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
ds4=xr.open_dataset(dir+'neusa_ep_days_stats_var95const_station95.nc')
ar_yesno4=ds4['ar_yesno'].values.tolist()
print(len(ar_yesno4))
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['ivt_tc_remnants_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['ar_tc_remnant_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()

ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
print(len(dates))

dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
print(dates)
years=[x.year for x in dates_unique]
print(years)
yrs_neusa=np.arange(1979,2020,1)

indices=np.arange(169)

#days with prior TC influences
dates_remote_tc_inf=[dt.datetime(1979,9,21,6),dt.datetime(1987,9,9,6),dt.datetime(1989,9,19,6),
                     dt.datetime(1996,9,15,6),dt.datetime(2002,9,16,6),
                     dt.datetime(2002,9,28,6),dt.datetime(2003,9,2,6),dt.datetime(2003,9,4,6),
                     dt.datetime(2005,9,1,6),dt.datetime(2005,10,7,6),dt.datetime(2005,10,8,6),
                     dt.datetime(2005,10,9,6),dt.datetime(2005,10,15,6),dt.datetime(2009,11,12,6),
                     dt.datetime(2010,10,1,6),dt.datetime(2011,9,7,6),dt.datetime(2011,9,8,6),
                     dt.datetime(2015,9,30,6),dt.datetime(2015,10,1,6),dt.datetime(2017,10,29,6),
                     dt.datetime(2017,10,30,6),dt.datetime(2018,9,9,6),dt.datetime(2018,9,10,6),
                     dt.datetime(2018,9,19,6)]
remote_tc_inf_yesno4=[]
for i in range(len(dates_unique)):
    if dates_unique[i] in dates_remote_tc_inf:
        remote_tc_inf_yesno4.append(1)
    else:
        remote_tc_inf_yesno4.append(0)
print(sum(remote_tc_inf_yesno4))

zipped_pure_ar=list(zip(indices,ar_yesno4))
zipped_tc_linked_ar=list(zip(indices,tc_linked_ar_yesno4))
zipped_tc_remnant_linked_ar=list(zip(indices,tc_remnant_linked_ar_yesno4))
zipped_pure_tc=list(zip(indices,tc_yesno4))
zipped_tc_ar_combo=list(zip(indices,tc_ar_combo_yesno4))
zipped_tc_remnant_ar_combo=list(zip(indices,tc_remnant_ar_combo_yesno4))
zipped_tc_remnants=list(zip(indices,tc_remnants_yesno4))
zipped_pure_extreme_ivt=list(zip(indices,ivt_yesno4))
zipped_tc_linked_extreme_ivt=list(zip(indices,tc_linked_ivt_yesno4))
zipped_tc_remnant_linked_extreme_ivt=list(zip(indices,tc_remnant_linked_ivt_yesno4))
zipped_other=list(zip(indices,other_yesno4))
zipped_remote=list(zip(indices,remote_tc_inf_yesno4))

indices_pure_ar=[x[0] for x in zipped_pure_ar if x[1]==1]
indices_tc_linked_ar=[x[0] for x in zipped_tc_linked_ar if x[1]==1]
indices_tc_remnant_linked_ar=[x[0] for x in zipped_tc_remnant_linked_ar if x[1]==1]
indices_pure_tc=[x[0] for x in zipped_pure_tc if x[1]==1]
indices_tc_ar_combo=[x[0] for x in zipped_tc_ar_combo if x[1]==1]
indices_tc_remnant_ar_combo=[x[0] for x in zipped_tc_remnant_ar_combo if x[1]==1]
indices_tc_remnants=[x[0] for x in zipped_tc_remnants if x[1]==1]
indices_pure_extreme_ivt=[x[0] for x in zipped_pure_extreme_ivt if x[1]==1]
indices_tc_linked_extreme_ivt=[x[0] for x in zipped_tc_linked_extreme_ivt if x[1]==1]
indices_tc_remnant_linked_extreme_ivt=[x[0] for x in zipped_tc_remnant_linked_extreme_ivt if x[1]==1]
indices_other=[x[0] for x in zipped_other if x[1]==1]
indices_remote=[x[0] for x in zipped_remote if x[1]==1]

print('PURE AR: '+str(indices_pure_ar))
print('TC-LINKED AR: '+str(indices_tc_linked_ar))
print('TC REMNANT-LINKED AR: '+str(indices_tc_remnant_linked_ar))
print('PURE TC: '+str(indices_pure_tc))
print('TC - AR COMBO: '+str(indices_tc_ar_combo))
print('TC REMNANT - AR COMBO: '+str(indices_tc_remnant_ar_combo))
print('TC REMNANTS: '+str(indices_tc_remnants))
print('PURE EXTREME IVT: '+str(indices_pure_extreme_ivt))
print('TC-LINKED EXTREME IVT: '+str(indices_tc_linked_extreme_ivt))
print('TC REMNANT-LINKED EXTREME IVT: '+str(indices_tc_remnant_linked_extreme_ivt))
print('UNSPECIFIED: '+str(indices_other))

count_tc_days=0
for i in range(len(ar_yesno4)):
    if tc_yesno4[i]==1 or tc_linked_ar_yesno4[i]==1 or tc_linked_ivt_yesno4[i]==1 or tc_remnant_ar_combo_yesno4[i]==1 or tc_ar_combo_yesno4[i]==1 or tc_remnants_yesno4[i]==1 or remote_tc_inf_yesno4[i]==1 or tc_remnant_linked_ivt_yesno4[i]==1 or tc_remnant_linked_ar_yesno4[i]==1:
        count_tc_days=count_tc_days+1

print('COUNT_TC_DAYS')
print(count_tc_days)
print([x for x in np.arange(169) if x not in indices_pure_ar+indices_tc_linked_ar+indices_tc_remnant_linked_ar+indices_pure_tc+indices_tc_ar_combo+indices_tc_remnant_ar_combo+indices_tc_remnants+indices_pure_extreme_ivt+indices_tc_linked_extreme_ivt+indices_tc_remnant_linked_extreme_ivt+indices_other])
print(len(indices_pure_ar)+len(indices_tc_linked_ar)+len(indices_tc_remnant_linked_ar)+len(indices_pure_tc)+len(indices_tc_ar_combo)+len(indices_tc_remnant_ar_combo)+len(indices_tc_remnants)+len(indices_pure_extreme_ivt)+len(indices_tc_linked_extreme_ivt)+len(indices_tc_remnant_linked_extreme_ivt)+len(indices_other))

In [ ]:
ds1=xr.open_dataset(dir+'station_numbers_95.nc')
stations_unique=ds1.stations.values.tolist()
print(ds1)
thresholds=ds1.thresholds_annual_99.values.tolist()
zipped_stations=list(zip(stations_unique,thresholds))

In [ ]:
yrs=np.arange(1979,2020,1)
zipped=list(zip(dates,obs,lats,lons,stations))
zipped_coords=list(zip(lats,lons))
coords_unique=list(set(zipped_coords))
mean_obs_day_list=[]
max_obs_day_list=[]
mean_obs_measurable_day_list=[]
for i in range(len(dates_unique)):
    print(i)
    date=dates_unique[i]
    select_day=[x for x in zipped if x[0]==date]# and x[2]>42]
    obs_day=[x[1] for x in select_day]
    mean_obs_day=float(sum(obs_day))/float(len(select_day))
    mean_obs_day_list.append(mean_obs_day)
    max_obs_day_list.append(max(obs_day))
    
    #want to define mean of measurable precipitation
    obs_measurable_day=[x[1] for x in select_day if x[1]>=1]
    mean_obs_measurable_day=float(sum(obs_measurable_day))/float(len(obs_measurable_day))
    mean_obs_measurable_day_list.append(mean_obs_measurable_day)

In [ ]:
print(len(dates_unique))
event_indices_array=[]
event_lengths_list=[]
event_years_list=[]
event_precip_totals_list=[]
event_start_dates=[]
event_end_dates=[]
event_wtypes_array=[]
event_precip_maxes_list=[]
event_measurable_mean_list=[]
event_extreme_obs_list=[]
for i in range(len(dates_unique)):
    print(i)
    year=dates_unique[i].year
    mean_obs=mean_obs_day_list[i]
    
    data_date=[x for x in zipped if x[0]==dates_unique[i]]
    
    ar_yesno=ar_yesno4[i]
    tc_linked_ar_yesno=tc_linked_ar_yesno4[i]
    tc_remnant_linked_ar_yesno=tc_remnant_linked_ar_yesno4[i]
    tc_yesno=tc_yesno4[i]
    tc_ar_combo_yesno=tc_ar_combo_yesno4[i]
    tc_remnant_ar_combo_yesno=tc_remnant_ar_combo_yesno4[i]
    tc_remnants_yesno=tc_remnants_yesno4[i]
    pure_extreme_ivt_yesno=ivt_yesno4[i]
    tc_linked_extreme_ivt_yesno=tc_linked_ivt_yesno4[i]
    tc_remnant_linked_extreme_ivt_yesno=tc_remnant_linked_ivt_yesno4[i]
    unspecified_yesno=other_yesno4[i]
    remote_yesno=remote_tc_inf_yesno4[i]
    
    if i==0:
        extreme_obs_sum=0
        event_length=1
        event_indices_list=[0]
        event_precip=mean_obs_day_list[0]
        measurable_precip=mean_obs_measurable_day_list[0]
        event_start_dates.append(dates_unique[0])
        data_date_list=data_date
        
        if ar_yesno==1:
            event_wtypes_list=['Pure AR']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_linked_ar_yesno==1:
            event_wtypes_list=['TC-linked AR']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_remnant_linked_ar_yesno==1:
            event_wtypes_list=['TC remnant-linked AR']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_yesno==1:
            event_wtypes_list=['Pure TC']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_ar_combo_yesno==1:
            event_wtypes_list=['TC - AR combination']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_remnant_ar_combo_yesno==1:
            event_wtypes_list=['TC remnant - AR combination']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_remnants_yesno==1:
            event_wtypes_list=['TC remnants']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if pure_extreme_ivt_yesno==1:
            event_wtypes_list=['Pure extreme IVT']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_linked_extreme_ivt_yesno==1:
            event_wtypes_list=['TC-linked extreme IVT']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if tc_remnant_linked_extreme_ivt_yesno==1:
            event_wtypes_list=['TC remnant-linked extreme IVT']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        if unspecified_yesno==1:
            event_wtypes_list=['Unspecified']
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
        
    else:
        if dates_unique[i]-dates_unique[i-1]==dt.timedelta(days=1):
            event_length=event_length+1
            event_indices_list.append(i)
            data_date_list=data_date_list+data_date
            if ar_yesno==1:
                event_wtypes_list.append('Pure AR')
            if tc_linked_ar_yesno==1:
                event_wtypes_list.append('TC-linked AR')
            if tc_remnant_linked_ar_yesno==1:
                event_wtypes_list.append('TC remnant-linked AR')
            if tc_yesno==1:
                event_wtypes_list.append('Pure TC')
            if tc_ar_combo_yesno==1:
                event_wtypes_list.append('TC - AR combination')
            if tc_remnant_ar_combo_yesno==1:
                event_wtypes_list.append('TC remnant - AR combination')
            if tc_remnants_yesno==1:
                event_wtypes_list.append('TC remnants')
            if pure_extreme_ivt_yesno==1:
                event_wtypes_list.append('Pure extreme IVT')
            if tc_linked_extreme_ivt_yesno==1:
                event_wtypes_list.append('TC-linked extreme IVT')
            if tc_remnant_linked_extreme_ivt_yesno==1:
                event_wtypes_list.append('TC remnant-linked extreme IVT')
            if unspecified_yesno==1:
                event_wtypes_list.append('Unspecified')
            #if remote_yesno==1:
            #    event_wtypes_list.append('Prior TC influence')
            event_precip=event_precip+mean_obs_day_list[i]
            
            measurable_precip=measurable_precip+mean_obs_measurable_day_list[i]
            
            if i==len(dates_unique)-1:
                event_lengths_list.append(event_length)
                event_indices_array.append(event_indices_list)
                event_wtypes_list=list(set(event_wtypes_list))
                event_wtypes_array.append(event_wtypes_list)
                event_years_list.append(year)
                event_precip_totals_list.append(event_precip)
                event_measurable_mean_list.append(measurable_precip)
                event_end_dates.append(dates_unique[i])
                
                data_date_list=data_date_list+data_date
                total_precip_list=[]
                measurable_precip_list=[]
                for k in range(len(stations_unique)):
                    select_precip=[x[1] for x in data_date_list if x[4]==stations_unique[k]]
                    total_precip=sum(select_precip)
                    total_precip_list.append(total_precip)
                    
                    threshold=[x[1]/10. for x in zipped_stations if x[0]==stations_unique[k]][0]
                    extreme_precip=[x for x in select_precip if x>=threshold]
                    extreme_obs_sum=extreme_obs_sum+len(extreme_precip)
                    
                max_station_precip=max(total_precip_list)
                event_precip_maxes_list.append(max_station_precip)
                
        if dates_unique[i]-dates_unique[i-1]!=dt.timedelta(days=1):
            event_lengths_list.append(event_length)
            event_indices_array.append(event_indices_list)
            event_wtypes_list=list(set(event_wtypes_list))
            event_wtypes_array.append(event_wtypes_list)
            event_years_list.append(year)
            event_precip_totals_list.append(event_precip)
            event_measurable_mean_list.append(measurable_precip)
            
            total_precip_list=[]
            for k in range(len(stations_unique)):
                select_precip=[x[1] for x in data_date_list if x[4]==stations_unique[k]]
                total_precip=sum(select_precip)
                total_precip_list.append(total_precip)
                threshold=[x[1]/10. for x in zipped_stations if x[0]==stations_unique[k]][0]
                extreme_precip=[x for x in select_precip if x>=threshold]
                extreme_obs_sum=extreme_obs_sum+len(extreme_precip)
            event_extreme_obs_list.append(extreme_obs_sum)
            max_station_precip=max(total_precip_list)
            event_precip_maxes_list.append(max_station_precip)
           
            event_length=1
            event_indices_list=[i]
            measurable_precip=0
            extreme_obs_sum=0
    
            data_date_list=data_date
                
            if ar_yesno==1:
                event_wtypes_list=['Pure AR']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_linked_ar_yesno==1:
                event_wtypes_list=['TC-linked AR']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_remnant_linked_ar_yesno==1:
                event_wtypes_list=['TC remnant-linked AR']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_yesno==1:
                event_wtypes_list=['Pure TC']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_ar_combo_yesno==1:
                event_wtypes_list=['TC - AR combination']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_remnant_ar_combo_yesno==1:
                event_wtypes_list=['TC remnant - AR combination']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_remnants_yesno==1:
                event_wtypes_list=['TC remnants']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if pure_extreme_ivt_yesno==1:
                event_wtypes_list=['Pure extreme IVT']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_linked_extreme_ivt_yesno==1:
                event_wtypes_list=['TC-linked extreme IVT']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if tc_remnant_linked_extreme_ivt_yesno==1:
                event_wtypes_list=['TC remnant-linked extreme IVT']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
            if unspecified_yesno==1:
                event_wtypes_list=['Unspecified']
                #if remote_yesno==1:
                #    event_wtypes_list.append('Prior TC influence')
                    
            
                
            event_precip=mean_obs_day_list[i]
            measurable_precip=measurable_precip+mean_obs_measurable_day_list[i]
            
            event_start_dates.append(dates_unique[i])
            event_end_dates.append(dates_unique[i-1])
            if i==len(dates_unique)-1:
                event_lengths_list.append(event_length)
                event_indices_array.append(event_indices_list)
                event_wtypes_list=list(set(event_wtypes_list))
                event_wtypes_array.append(event_wtypes_list)
                
                data_date_list=data_date
                total_precip_list=[]
                measurable_precip_list=[]
                for k in range(len(stations_unique)):
                    select_precip=[x[1] for x in data_date_list if x[4]==stations_unique[k]]
                    total_precip=sum(select_precip)
                    total_precip_list.append(total_precip)
                    threshold=[x[1]/10. for x in zipped_stations if x[0]==stations_unique[k]][0]
                    extreme_precip=[x for x in select_precip if x>=threshold]
                    extreme_obs_sum=extreme_obs_sum+len(extreme_precip)
                max_station_precip=max(total_precip_list)
                event_precip_maxes_list.append(max_station_precip)
                event_extreme_obs_list.append(extreme_obs_sum)
                
                event_years_list.append(year)
                event_precip_totals_list.append(event_precip)
                event_measurable_mean_list.append(measurable_precip)
                event_end_dates.append(dates_unique[i])
print(event_lengths_list)
print(event_indices_array)
print(event_years_list)
print(event_wtypes_array)
print(event_precip_maxes_list)

daily_measurable_mean_list=[float(x)/float(y) for x,y in zip(event_measurable_mean_list,event_lengths_list)]
daily_extreme_reports_list=[float(x)/float(y) for x,y in zip(event_extreme_obs_list,event_lengths_list)]

event_start_dates=[str(x)[0:10] for x in event_start_dates]
event_end_dates=[str(x)[0:10] for x in event_end_dates]
print(event_start_dates)
print(event_end_dates)

In [ ]:
print(len(event_wtypes_array))
tc_types=['Pure TC','TC remnants','TC-linked AR','TC remnant-linked AR','TC-linked extreme IVT','TC remnant-linked extreme IVT','TC - AR combination','TC remnant - AR combination','Prior TC influence']
count_tc=0
for i in range(len(event_wtypes_array)):
    types_list=event_wtypes_array[i]
    intersect=[x for x in types_list if x in tc_types]
    if len(intersect)>0:
        count_tc=count_tc+1
print(count_tc)

In [ ]:
print(daily_measurable_mean_list)
print(len(daily_measurable_mean_list))
print(daily_extreme_reports_list)
print(len(daily_extreme_reports_list))

In [ ]:
zipped_years_rain=list(zip(event_years_list,event_precip_totals_list,event_lengths_list,event_start_dates,event_end_dates,event_wtypes_array,event_precip_maxes_list,daily_measurable_mean_list,daily_extreme_reports_list))
freq_yrs_list=[]
precip_yrs_list=[]
length_yrs_list=[]
rates_yrs_list=[]
total_precip_list=[]
for i in yrs:
    select=[x for x in zipped_years_rain if x[0]==i]
    #select=[x for x in select if 'Pure AR' in x[5] or 'TC-linked AR' in x[5] or 'TC remnant-linked AR' in x[5]]
    #select=[x for x in select if 'Pure TC' in x[5] or 'TC - AR combination' in x[5] or 'TC remnant - AR combination' in x[5] or 'TC remnants' in x[5]]
    freq=len(select)
    freq_yrs_list.append(freq)
    precip_totals=[x[1] for x in select]
    lengths=[x[2] for x in select]
    total_precip=sum(precip_totals)
    if len(precip_totals)>0:
        mean_precip_total=float(sum(precip_totals))/float(len(precip_totals))
        mean_length=float(sum(lengths))/float(len(lengths))
        
        daily_precip_rates=[x/y for x,y in zip(precip_totals,lengths)]
        mean_daily_rate=float(sum(daily_precip_rates))/float(len(daily_precip_rates))
        
        
    else:
        mean_precip_total=np.nan
        mean_length=np.nan
        mean_daily_rate=np.nan
    precip_yrs_list.append(mean_precip_total)
    length_yrs_list.append(mean_length)
    rates_yrs_list.append(mean_daily_rate)
    total_precip_list.append(total_precip)

In [ ]:
print(len(zipped_years_rain))
print([x[0] for x in zipped_years_rain if x[2]>2])

one_day_plus=[x[0] for x in zipped_years_rain]
two_day_plus=[x[0] for x in zipped_years_rain if x[2]>1]
three_day_plus=[x[0] for x in zipped_years_rain if x[2]>2]

one_plus_list=[]
two_plus_list=[]
three_plus_list=[]
for i in range(len(yrs)):
    year=yrs[i]
    select_one=[x for x in one_day_plus if x==year]
    select_two=[x for x in two_day_plus if x==year]
    select_three=[x for x in three_day_plus if x==year]
    
    one_plus_list.append(len(select_one))
    two_plus_list.append(len(select_two))
    three_plus_list.append(len(select_three))

    
mk=mk_test(one_plus_list,alpha=0.05)
line=st.linregress(yrs,one_plus_list)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label='All events ('+eq_1+') | '+mk_str


mk=mk_test(two_plus_list,alpha=0.05)
line=st.linregress(yrs,two_plus_list)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label1='2+ day events ('+eq_1+') | '+mk_str


mk=mk_test(three_plus_list,alpha=0.05)
line=st.linregress(yrs,three_plus_list)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label2='3+ day events ('+eq_1+') | '+mk_str

fig=plt.figure(figsize=(16,10))
ax=plt.subplot(1,1,1)
ax.bar(yrs,one_plus_list,fill=False,color='k',label='All events')
ax.bar(yrs,two_plus_list,color='b',alpha=0.3,label='2+ day events')
ax.bar(yrs,three_plus_list,color='darkblue',label='3+ day events')
ax.tick_params(labelsize=16)
ax.set_xlabel('Years',fontsize=17)
ax.set_ylabel('Number of events',fontsize=17)
ax.set_title('Events by duration',fontsize=26)
plt.legend(loc='upper left',fontsize=16)
ax.set_ylim(0,8)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_event_duration_series.png')

In [ ]:
a=[x[0] for x in zipped_years_rain if 'TC-linked AR' in x[5] or 'TC remnant-linked AR' in x[5] or 'TC remnant - AR combination' in x[5] or 'TC - AR combination' in x[5]]
print(a)

In [ ]:
years_complex=[]
durations_complex=[]
precip_complex=[]
complex_yesno=[]
precip_all=[]
tc_yesno6=[]
for i in range(len(zipped_years_rain)):
    wtypes_get=zipped_years_rain[i][5]
    ar_yes=0
    tc_yes=0
    for j in range(len(wtypes_get)):
        if 'AR' in wtypes_get[j]:
            ar_yes=1
        if 'TC' in wtypes_get[j]:
            tc_yes=1
    if ar_yes==1 and tc_yes==1:
        years_complex.append(zipped_years_rain[i][0])
        durations_complex.append(zipped_years_rain[i][2])
        precip_complex.append(zipped_years_rain[i][1])
        complex_yesno.append(1)
    else:
        complex_yesno.append(0)
    if tc_yes==1:
        tc_yesno6.append(1)
    else:
        tc_yesno6.append(0)
        
    precip_all.append(zipped_years_rain[i][1])
print(years_complex)

freqs_complex=[]
days_complex=[]
precip_totals_complex=[]
zipped_years_days=list(zip(years_complex,durations_complex,precip_complex))
for i in range(len(yrs)):
    year=yrs[i]
    select_complex=[x for x in years_complex if x==year]
    select_days_complex=[x[1] for x in zipped_years_days if x[0]==year]
    select_precip_complex=[x[2] for x in zipped_years_days if x[0]==year]
    freqs_complex.append(len(select_complex))
    days_complex.append(sum(select_days_complex))
    precip_totals_complex.append(sum(select_precip_complex))
    
zipped_precip_all=list(zip(complex_yesno,precip_all,tc_yesno6))
sorted_precip_all=sorted(zipped_precip_all,key=lambda x:x[1])[::-1]
print(sorted_precip_all)

precip_bins=np.arange(0,100,10)
num_complex_list=[]
num_all_list=[]
num_tc_list=[]
for i in range(len(precip_bins)-1):
    select_precip_all=[x for x in zipped_precip_all if precip_bins[i]<=x[1]<precip_bins[i+1]]
    select_precip_complex=[x for x in zipped_precip_all if x[0]==1 and precip_bins[i]<=x[1]<precip_bins[i+1]]
    select_precip_tc=[x for x in zipped_precip_all if x[2]==1 and precip_bins[i]<=x[1]<precip_bins[i+1]]
    num_all=len(select_precip_all)
    num_complex=len(select_precip_complex)
    num_tc=len(select_precip_tc)
    num_complex_list.append(float(num_complex)/float(num_all))
    num_all_list.append(num_all)
    num_tc_list.append(float(num_tc)/float(num_all))


fig=plt.figure(figsize=(16,10))
ax=plt.subplot(1,1,1)
#ax.bar(yrs,freqs_complex,fill=False,color='k',label='All events')
#ax.bar(yrs,days_complex,color='b',alpha=0.3,label='All events')

ax.plot(precip_bins[:-1]+5,num_all_list,color='k',linewidth=2.0,label='Total number of events')
ax.set_ylim(0,max(num_all_list)*1.3)
ax.tick_params(labelsize=16)
ax.set_ylabel('Number of events',fontsize=17,labelpad=15)
ax.set_xlabel('Mean total precipitation (mm)',fontsize=17)
ax2=ax.twinx()
ax2.bar(precip_bins[:-1]+5,num_complex_list,color='b',width=8,alpha=0.3,label='Fraction with AR and TC influences')
ax2.bar(precip_bins[:-1]+5,num_tc_list,fill=False,color='k',width=8,label='Fraction with TC influences')

print('LISTS')
print(len([x for x in zipped_precip_all if x[0]==1]))
print(len([x for x in zipped_precip_all if x[2]==1]))
ax2.tick_params(labelsize=16)

ax2.set_ylabel('Fraction of events',fontsize=17,labelpad=15)
ax2.set_title('TC influences on EP events',fontsize=26)
#plt.legend(loc='upper left',fontsize=16)
ax2.set_ylim(0,max(num_complex_list)*1.2)

#lns=lns1+lns2+lns3
lines, labels = ax.get_legend_handles_labels()
print(lines,labels)
lines2, labels2 = ax2.get_legend_handles_labels()
lines2=lines2[::-1]
labels2=labels2[::-1]
ax2.legend(lines + lines2, labels + labels2, loc='upper left',fontsize=16)

for i in range(len(num_all_list)):
    if i==0:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))+0.06,str(num_all_list[i]),fontsize=16)
    if i==1:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))+-0.04,str(num_all_list[i]),fontsize=16)
    if i==2:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))-0.01,str(num_all_list[i]),fontsize=16)
    if i==3:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))-0.02,str(num_all_list[i]),fontsize=16)
    if i==4:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))+0.01,str(num_all_list[i]),fontsize=16)
    if i==5:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))+0.01,str(num_all_list[i]),fontsize=16)
    if i==6:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))+0.02,str(num_all_list[i]),fontsize=16)
    if i==7:
        ax2.text(precip_bins[i]+4.5,num_all_list[i]/(max(num_all_list))+0.02,str(num_all_list[i]),fontsize=16)
    if i==8:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))+0.02,str(num_all_list[i]),fontsize=16)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_precip_intensity_wtype_influences1.png')

In [ ]:
years_complex=[]
durations_complex=[]
precip_complex=[]
complex_yesno=[]
precip_all=[]
tc_yesno6=[]
for i in range(len(zipped_years_rain)):
    wtypes_get=zipped_years_rain[i][5]
    ar_yes=0
    tc_yes=0
    for j in range(len(wtypes_get)):
        if 'AR' in wtypes_get[j]:
            ar_yes=1
        if 'TC' in wtypes_get[j]:
            tc_yes=1
    if ar_yes==1 and tc_yes==1:
        years_complex.append(zipped_years_rain[i][0])
        durations_complex.append(zipped_years_rain[i][2])
        precip_complex.append(zipped_years_rain[i][1])
        complex_yesno.append(1)
    else:
        complex_yesno.append(0)
    if ar_yes==1:
        tc_yesno6.append(1)
    else:
        tc_yesno6.append(0)
        
    precip_all.append(zipped_years_rain[i][1])
print(years_complex)

freqs_complex=[]
days_complex=[]
precip_totals_complex=[]
zipped_years_days=list(zip(years_complex,durations_complex,precip_complex))
for i in range(len(yrs)):
    year=yrs[i]
    select_complex=[x for x in years_complex if x==year]
    select_days_complex=[x[1] for x in zipped_years_days if x[0]==year]
    select_precip_complex=[x[2] for x in zipped_years_days if x[0]==year]
    freqs_complex.append(len(select_complex))
    days_complex.append(sum(select_days_complex))
    precip_totals_complex.append(sum(select_precip_complex))
    
zipped_precip_all=list(zip(complex_yesno,precip_all,tc_yesno6))
sorted_precip_all=sorted(zipped_precip_all,key=lambda x:x[1])[::-1]
print(sorted_precip_all)

precip_bins=np.arange(0,100,10)
num_complex_list=[]
num_all_list=[]
num_tc_list=[]
for i in range(len(precip_bins)-1):
    select_precip_all=[x for x in zipped_precip_all if precip_bins[i]<=x[1]<precip_bins[i+1]]
    select_precip_complex=[x for x in zipped_precip_all if x[0]==1 and precip_bins[i]<=x[1]<precip_bins[i+1]]
    select_precip_tc=[x for x in zipped_precip_all if x[2]==1 and precip_bins[i]<=x[1]<precip_bins[i+1]]
    num_all=len(select_precip_all)
    num_complex=len(select_precip_complex)
    num_tc=len(select_precip_tc)
    num_complex_list.append(float(num_complex)/float(num_all))
    num_all_list.append(num_all)
    num_tc_list.append(float(num_tc)/float(num_all))

    
fig=plt.figure(figsize=(16,10))
ax=plt.subplot(1,1,1)
#ax.bar(yrs,freqs_complex,fill=False,color='k',label='All events')
#ax.bar(yrs,days_complex,color='b',alpha=0.3,label='All events')

ax.plot(precip_bins[:-1]+5,num_all_list,color='k',linewidth=2.0,label='Total number of events')
ax.set_ylim(0,max(num_all_list)*1.3)
ax.tick_params(labelsize=16)
ax.set_ylabel('Number of events',fontsize=17,labelpad=15)
ax.set_xlabel('Total precipitation',fontsize=17)
ax2=ax.twinx()
ax2.bar(precip_bins[:-1]+5,num_complex_list,color='b',width=8,alpha=0.3,label='Fraction with AR and TC influences')
ax2.bar(precip_bins[:-1]+5,num_tc_list,fill=False,color='k',width=8,label='Fraction with AR influences')


ax2.tick_params(labelsize=16)

ax2.set_ylabel('Fraction of events',fontsize=17,labelpad=15)
ax2.set_title('AR influences by event precipitation',fontsize=26)
#plt.legend(loc='upper left',fontsize=16)
ax2.set_ylim(0,max(num_complex_list)*1.2)

#lns=lns1+lns2+lns3
lines, labels = ax.get_legend_handles_labels()
print(lines,labels)
lines2, labels2 = ax2.get_legend_handles_labels()
print(labels2)
lines2=lines2[::-1]
labels2=labels2[::-1]
ax2.legend(lines + lines2, labels + labels2, loc='upper left',fontsize=16)

for i in range(len(num_all_list)):
    if i==0:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))+0.06,str(num_all_list[i]),fontsize=16)
    if i==1:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))+-0.04,str(num_all_list[i]),fontsize=16)
    if i==2:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))-0.01,str(num_all_list[i]),fontsize=16)
    if i==3:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))-0.02,str(num_all_list[i]),fontsize=16)
    if i==4:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))+0.01,str(num_all_list[i]),fontsize=16)
    if i==5:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))+0.01,str(num_all_list[i]),fontsize=16)
    if i==6:
        ax2.text(precip_bins[i]+5,num_all_list[i]/(max(num_all_list))+0.02,str(num_all_list[i]),fontsize=16)
    if i==7:
        ax2.text(precip_bins[i]+4.5,num_all_list[i]/(max(num_all_list))+0.02,str(num_all_list[i]),fontsize=16)
    if i==8:
        ax2.text(precip_bins[i]+4,num_all_list[i]/(max(num_all_list))+0.02,str(num_all_list[i]),fontsize=16)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_precip_intensity_wtype_influences2.png')

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm

from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
mk=mk_test(freq_yrs_list,alpha=0.05)
line=st.linregress(yrs,freq_yrs_list)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label='EP event frequency ('+eq_1+')\n'+mk_str

slope=line[0]
intercept=line[1]
fig=plt.figure(figsize=(16,8))
ax=plt.subplot(1,1,1)
ax.bar(yrs,freq_yrs_list,color='b',label=label,zorder=20)
ax.plot(yrs,[slope*x+intercept for x in yrs],color='darkblue',linewidth=3.0,zorder=21)

ax.set_xlabel('Year',fontsize=22)
ax.set_ylabel('Number of Events',fontsize=22)
ax.set_title('Event frequency',fontsize=32)
ax.legend(loc='upper left',fontsize=22)
ax.set_ylim(0,max(freq_yrs_list)*1.3)
ax.tick_params(labelsize=22)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_event_perspective_1.png')

In [ ]:
zipped_precip=list(zip(yrs,total_precip_list))
select_yrs=[x[0] for x in zipped_precip if x[1]>=0]
select_precip=[x[1] for x in zipped_precip if x[1]>=0]

mk=mk_test(total_precip_list,alpha=0.05)
line=st.linregress(select_yrs,select_precip)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label='Total precipitation ('+eq_1+')\n'+mk_str

slope=line[0]
intercept=line[1]
fig=plt.figure(figsize=(16,8))
ax=plt.subplot(1,1,1)
ax.bar(yrs,total_precip_list,color='g',label=label,zorder=20)
ax.plot(yrs,[slope*x+intercept for x in yrs],color='darkgreen',linewidth=3.0,zorder=21)

ax.set_xlabel('Year',fontsize=22)
ax.set_ylabel('Total precipitation',fontsize=22)
ax.set_title('Fall total EP day precipitation',fontsize=32)
ax.legend(loc='upper left',fontsize=22)
ax.set_ylim(0,max(total_precip_list)*1.3)
ax.tick_params(labelsize=22)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_event_perspective_2.png')

In [ ]:
zipped_length=list(zip(yrs,length_yrs_list))
select_yrs=[x[0] for x in zipped_length if x[1]>=0]
select_length=[x[1] for x in zipped_length if x[1]>=0]

mk=mk_test(length_yrs_list,alpha=0.05)
line=st.linregress(select_yrs,select_length)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label='EP event duration ('+eq_1+') '

slope=line[0]
intercept=line[1]
fig=plt.figure(figsize=(16,8))
ax=plt.subplot(1,1,1)
ax.bar(yrs,length_yrs_list,color='r',label=label,zorder=20)
ax.plot(yrs,[slope*x+intercept for x in yrs],color='darkred',linewidth=3.0,zorder=21)

ax.set_xlabel('Year',fontsize=22)
ax.set_ylabel('Mean duration (days)',fontsize=22)
ax.set_title('Event duration',fontsize=32)
ax.legend(loc='upper left',fontsize=22)
ax.set_ylim(0,max(length_yrs_list)*1.3)
ax.tick_params(labelsize=22)

plt.show()

In [ ]:
fig.savefig(dir+'neusa_event_perspective_3.png')

In [ ]:
zipped_rate=list(zip(yrs,rates_yrs_list))
select_yrs=[x[0] for x in zipped_rate if x[1]>=0]
select_rate=[x[1] for x in zipped_rate if x[1]>=0]

mk=mk_test(rates_yrs_list,alpha=0.05)
line=st.linregress(select_yrs,select_rate)

if mk[0] in ['increasing','decreasing']:
    mk_str='MK-test = significant'
else:
    mk_str='MK-test = insignificant'
    
if line[3]<=0.05:
    line_str='Lin-reg = significant'
else:
    line_str='Lin-reg = insignificant'
if line[1]<0:
    eq_1='y='+str(round(line[0],3))+'x-'+str(round(abs(line[1]),2))+', p='+str(round(line[3],4))
else:
    eq_1='y='+str(round(line[0],3))+'x+'+str(round(line[1],2))+', p='+str(round(line[3],4))
label='EP event daily precipitation rate ('+eq_1+') '

slope=line[0]
intercept=line[1]
fig=plt.figure(figsize=(16,8))
ax=plt.subplot(1,1,1)
ax.bar(yrs,rates_yrs_list,color='orange',label=label,zorder=20)
ax.plot(yrs,[slope*x+intercept for x in yrs],color='saddlebrown',linewidth=3.0,zorder=21)

ax.set_xlabel('Year',fontsize=22)
ax.set_ylabel('Precipitation rate (mm/day)',fontsize=22)
ax.set_title('Event daily precipitation rate',fontsize=32)
ax.legend(loc='upper left',fontsize=22)
ax.set_ylim(0,max(rates_yrs_list)*1.3)
ax.tick_params(labelsize=22)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_event_perspective_4.png')

In [ ]:
#now list top event characteristics
sorted_by_precip=sorted(zipped_years_rain,key=lambda x:x[1])
print(sorted_by_precip[-10:])

In [ ]:
top_10_wtypes=[x[5] for x in sorted_by_precip[-10:]][::-1]
print(top_10_wtypes)
wtype_strings=[]
for i in range(len(top_10_wtypes)):
    wtype_set=top_10_wtypes[i]
    total_string=''
    for j in range(len(wtype_set)):
        if j<len(wtype_set)-1:
            total_string=total_string+wtype_set[j]+', '
        else:
            total_string=total_string+wtype_set[j]
    wtype_strings.append(total_string)
print(wtype_strings)
top_10_starting_dates=[str(x[3]) for x in sorted_by_precip[-10:][::-1]]
top_10_ending_dates=[str(x[4]) for x in sorted_by_precip[-10:][::-1]]
top_10_durations=[str(x[2]) for x in sorted_by_precip[-10:][::-1]]
top_10_precips=[str(round(x[1],2)) for x in sorted_by_precip[-10:][::-1]]
top_10_daily_measurable_means=[str(round(x[7],2)) for x in sorted_by_precip[-10:][::-1]]
top_10_extreme_reports=[str(round(x[8],2)) for x in sorted_by_precip[-10:][::-1]]

measurable_extremes=[x+' ['+str(y)+']' for x,y in zip(top_10_daily_measurable_means,top_10_extreme_reports)]

cell_text=[]
for i in range(10):
    row=[top_10_precips[i],top_10_durations[i],top_10_starting_dates[i],wtype_strings[i]]#measurable_extremes[i]=2
    cell_text.append(row)
print(cell_text)
columns=['Mean precipitation (mm)','Duration (days)','Starting date','Weather types']#'Intensity (mm/day) [# extreme]'=2
#cell_text=[top_10_precips,top_10_durations,top_10_starting_dates,top_10_ending_dates,top_10_wtypes]

from matplotlib.font_manager import FontProperties
fig, ax = plt.subplots() 
ax.set_axis_off() 
table=ax.table(cellText=cell_text,rowLabels=['']*10,colLabels=columns,cellLoc ='center',loc ='upper left')
table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(3.2, 1.5)
for (row, col), cell in table.get_celld().items():
    if (row == 0) or (col == -1):
        #cell.set_text_props(fontproperties=FontProperties(size=17))
        cell.get_text().set_fontsize(20)
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    else:
        cell.get_text().set_fontsize(14)
table.auto_set_column_width(col=list(range(4)))
ax.text(1.05,1.02,'Events with highest mean precipitation',fontsize=20)

In [ ]:
fig.savefig(dir+'neusa_event_perspective_100.png',bbox_inches="tight")

In [ ]:
sorted_by_precip_max=sorted(zipped_years_rain,key=lambda x:x[6])

top_10_wtypes=[x[5] for x in sorted_by_precip_max[-10:]][::-1]
print(top_10_wtypes)
wtype_strings=[]
for i in range(len(top_10_wtypes)):
    wtype_set=top_10_wtypes[i]
    total_string=''
    for j in range(len(wtype_set)):
        if j<len(wtype_set)-1:
            total_string=total_string+wtype_set[j]+', '
        else:
            total_string=total_string+wtype_set[j]
    wtype_strings.append(total_string)
print(wtype_strings)
top_10_starting_dates=[str(x[3]) for x in sorted_by_precip_max[-10:][::-1]]
top_10_ending_dates=[str(x[4]) for x in sorted_by_precip_max[-10:][::-1]]
top_10_durations=[str(x[2]) for x in sorted_by_precip_max[-10:][::-1]]
top_10_precips=[str(round(x[6],2)) for x in sorted_by_precip_max[-10:][::-1]]
top_10_daily_measurable_means=[str(round(x[7],2)) for x in sorted_by_precip_max[-10:][::-1]]
top_10_extreme_reports=[str(round(x[8],2)) for x in sorted_by_precip_max[-10:][::-1]]

measurable_extremes=[x+' ['+str(y)+']' for x,y in zip(top_10_daily_measurable_means,top_10_extreme_reports)]

cell_text=[]
for i in range(10):
    row=[top_10_precips[i],measurable_extremes[i],top_10_durations[i],top_10_starting_dates[i],wtype_strings[i]]#measurable_extremes[i]=2
    cell_text.append(row)
print(cell_text)
columns=['Maximum precipitation (mm)','Intensity (mm/day) [# extreme]','Duration (days)','Starting date','Weather types']#'Intensity (mm/day) [# extreme]'=2
#cell_text=[top_10_precips,top_10_durations,top_10_starting_dates,top_10_ending_dates,top_10_wtypes]

from matplotlib.font_manager import FontProperties
fig, ax = plt.subplots() 
ax.set_axis_off() 
table=ax.table(cellText=cell_text,rowLabels=['']*10,colLabels=columns,cellLoc ='center',loc ='upper left')
table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(3.2, 1.5)
for (row, col), cell in table.get_celld().items():
    if (row == 0) or (col == -1):
        #cell.set_text_props(fontproperties=FontProperties(size=17))
        cell.get_text().set_fontsize(20)
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    else:
        cell.get_text().set_fontsize(14)
table.auto_set_column_width(col=list(range(5)))#5
#ax.set_title('Top 10 events by mean total precipitation',fontsize=24)
ax.text(1.025,1.02,'Events with highest single station accumulation',fontsize=24)

In [ ]:
fig.savefig(dir+'neusa_event_perspective_101.png',bbox_inches="tight")

In [ ]:
sorted_by_precip_max=sorted(zipped_years_rain,key=lambda x:x[7])

top_10_wtypes=[x[5] for x in sorted_by_precip_max[-10:]][::-1]
print(top_10_wtypes)
wtype_strings=[]
for i in range(len(top_10_wtypes)):
    wtype_set=top_10_wtypes[i]
    total_string=''
    for j in range(len(wtype_set)):
        if j<len(wtype_set)-1:
            total_string=total_string+wtype_set[j]+', '
        else:
            total_string=total_string+wtype_set[j]
    wtype_strings.append(total_string)
print(wtype_strings)
top_10_starting_dates=[str(x[3]) for x in sorted_by_precip_max[-10:][::-1]]
top_10_ending_dates=[str(x[4]) for x in sorted_by_precip_max[-10:][::-1]]
top_10_durations=[str(x[2]) for x in sorted_by_precip_max[-10:][::-1]]
top_10_precips=[str(round(x[6],2)) for x in sorted_by_precip_max[-10:][::-1]]
top_10_daily_measurable_means=[str(round(x[7],2)) for x in sorted_by_precip_max[-10:][::-1]]
top_10_extreme_reports=[str(round(x[8],2)) for x in sorted_by_precip_max[-10:][::-1]]

measurable_extremes=[x+' ['+str(y)+']' for x,y in zip(top_10_daily_measurable_means,top_10_extreme_reports)]

cell_text=[]
for i in range(10):
    row=[top_10_precips[i],measurable_extremes[i],top_10_durations[i],top_10_starting_dates[i],wtype_strings[i]]
    cell_text.append(row)
print(cell_text)
columns=['Maximum precipitation (mm)','Intensity (mm/day) [# extreme]','Duration (days)','Starting date','Weather types']
#cell_text=[top_10_precips,top_10_durations,top_10_starting_dates,top_10_ending_dates,top_10_wtypes]

from matplotlib.font_manager import FontProperties
fig, ax = plt.subplots() 
ax.set_axis_off() 
table=ax.table(cellText=cell_text,rowLabels=['']*10,colLabels=columns,cellLoc ='center',loc ='upper left')
table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(3.2, 1.5)
for (row, col), cell in table.get_celld().items():
    if (row == 0) or (col == -1):
        #cell.set_text_props(fontproperties=FontProperties(size=17))
        cell.get_text().set_fontsize(20)
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    else:
        cell.get_text().set_fontsize(14)
table.auto_set_column_width(col=list(range(5)))
#ax.set_title('Top 10 events by mean total precipitation',fontsize=24)
ax.text(0.84,1.02,'Events with highest daily mean measurable precip',fontsize=20)

In [ ]:
fig.savefig(dir+'neusa_event_perspective_12a.png',bbox_inches="tight")

In [ ]:
#for each wtype - events containing wtype vs event length
#average total precip for events containing each wtype
str1='TC-linked extreme IVT, Unspecified'
if 'TC-linked extreme IVT' in str1:
    print('YES')

strings=['Pure AR','TC-linked AR','TC remnant-linked AR','Pure TC','TC - AR combination','TC remnant - AR combination','TC remnants','Pure extreme IVT','TC-linked extreme IVT','TC remnant-linked extreme IVT','Unspecified']
#for i in range(len(strings)):

indices_iterate=[0,1,3,4,5,6,7,10]
fig=plt.figure(figsize=(16,8))
for i in range(len(indices_iterate)):
    string=strings[indices_iterate[i]]
    lengths_wtype=[x[2] for x in zipped_years_rain if string in x[5]]
    x_arr=np.arange(1,5,1)
    dur_freq=[]
    for j in range(len(x_arr)):
        select_wtype=[x for x in lengths_wtype if x==x_arr[j]]
        dur_freq.append(len(select_wtype))
    
    ax=plt.subplot(1,8,i+1)
    ax.bar(x_arr,dur_freq,color='g',zorder=20)
    ax.set_xlabel('Event EP days',fontsize=12)
    if i==0:
        ax.set_ylabel('Number of events',fontsize=16)
    ax.set_title(string,fontsize=16)
    if i in [0,1,2,5,7]:
        ax.set_title(string,fontsize=16)
    if i==3:
        ax.set_title('TC - AR \n combination')
    if i==4:
        ax.set_title('TC remnant - \n AR combination')
    if i==6:
        ax.set_title('Pure \n extreme IVT')
    #ax.legend(loc='upper left',fontsize=18)
    #ax.set_ylim(0,max(dur_freq)*1.2)
    ax.set_ylim(0,28)
    ax.tick_params(labelsize=16)
    if i>0:
        ax.set_yticks([])
plt.subplots_adjust(wspace=0.02)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_event_duration_freqs_panel.png')

In [ ]:
mean_precips_list=[]
max_precips_list=[]
lengths_list=[]
strings_used=[]
sample_size_list=[]
for i in range(len(indices_iterate)):
    string=strings[indices_iterate[i]]
    precips_wtype=[x[1] for x in zipped_years_rain if string in x[5]]
    mean_precip=float(sum(precips_wtype))/float(len(precips_wtype))
    mean_precips_list.append(mean_precip)
    max_precips_wtype=[x[6] for x in zipped_years_rain if string in x[5]]
    max_precip_avg=float(sum(max_precips_wtype))/float(len(max_precips_wtype))
    max_precips_list.append(max_precip_avg)
    lengths_wtype=[x[2] for x in zipped_years_rain if string in x[5]]
    lengths_avg=float(sum(lengths_wtype))/float(len(lengths_wtype))
    lengths_list.append(lengths_avg)
    
    sample_size=len(precips_wtype)
    sample_size_list.append(sample_size)
    
    strings_used.append(string)
    
zipped_events=list(zip(strings_used,mean_precips_list,max_precips_list,lengths_list,sample_size_list))
sorted_events=sorted(zipped_events,key=lambda x:x[1])[::-1]

strings_used=[x[0] for x in sorted_events]
mean_precips_list=[x[1] for x in sorted_events]
max_precips_list=[x[2] for x in sorted_events]
lengths_list=[x[3] for x in sorted_events]
sample_size_list=[x[4] for x in sorted_events]

cell_text=[]
for i in range(len(strings_used)):
    row=[strings_used[i],sample_size_list[i],round(mean_precips_list[i],2),round(max_precips_list[i],2),round(lengths_list[i],2)]
    cell_text.append(row)
    
print(cell_text)
columns=['Events including...','Number of events','Mean precipitation (mm)','Maximum precipitation (mm)','Duration (days)']
rows=strings_used

from matplotlib.font_manager import FontProperties
fig, ax = plt.subplots() 
ax.set_axis_off() 
table=ax.table(cellText=cell_text,rowLabels=['']*8,colLabels=columns,cellLoc ='center',loc ='upper left')
table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(2.6, 1.5)
for (row, col), cell in table.get_celld().items():
    if (row == 0) or (col == -1):
        #cell.set_text_props(fontproperties=FontProperties(size=17))
        cell.get_text().set_fontsize(20)
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    else:
        cell.get_text().set_fontsize(14)
table.auto_set_column_width(col=list(range(5)))
ax.text(0.58,1.02,'Characteristics of events by included weather type',fontsize=18)

In [ ]:
fig.savefig(dir+'neusa_event_perspective_7.png',bbox_inches="tight")